In [2]:
import numpy as np
import torch
import lightkurve as lk

print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Lightkurve: {lk.__version__}")
# print(f"CUDA available: {torch.cuda.is_available()}")

lc = lk.search_lightcurve("Kepler-10", author="Kepler", cadence="long")
print(f"Found {len(lc)} light curves for Kepler-10.")

NumPy: 2.4.1
PyTorch: 2.9.1+cpu
Lightkurve: 2.5.1
Found 15 light curves for Kepler-10.


In [3]:
import pandas as pd

url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+*+from+ps+where+disc_facility+like+'%25Kepler%25'+and+default_flag=1&format=csv"
planets_df = pd.read_csv(url)

print(f"Found {len(planets_df)} confirmed Kepler planets")

Found 2783 confirmed Kepler planets


C:\Users\mikem\AppData\Local\Temp\ipykernel_16628\3084108069.py:4: DtypeWarning: Columns (3,184) have mixed types. Specify dtype option on import or set low_memory=False.
  planets_df = pd.read_csv(url)


In [4]:
import lightkurve as lk
import os

def download_lightcurve(kepid, save_dir="data/raw"):
    """Download Kepler light curve by KIC ID."""
    
    search = lk.search_lightcurve(
        f"KIC {kepid}",
        mission="Kepler",
        cadence="long"
    )
    
    if len(search) == 0:
        return None
    
    # Download all quarters, stitch together
    lc_collection = search.download_all()
    lc = lc_collection.stitch()
    
    # Save
    os.makedirs(save_dir, exist_ok=True)
    filepath = os.path.join(save_dir, f"kic_{kepid}.fits")
    lc.to_fits(filepath, overwrite=True)
    
    return lc